**RDF creation of tensile test data using TTO**

In this notebook, data originating from tensile tests performed to an S355 steel sheet is read in to create an exemplary RDF data file that uses the [Tensile Test Ontology (TTO) version 3.0](https://github.com/materialdigital/tensile-test-ontology) as semantic backbone. Hence, the tensile test data available in 'classic' test formats is transformed to RDF data by creating triples using the RDFlib library and some helper functions. Therefore, values tokenized from the CSV are allocated to concepts of the TTO developed in the frame of the PMD project. Since the TTO is strongly based on the PMD Core Ontology - PMDco (namespace: [https://w3id.org/pmd/co](https://w3id.org/pmd/co)) - the PMDco will also be parsed and semantic concepts this ontology is related to will also be used, accordingly. The tensile test data is taken from a dataset published openly available on [Zenodo](https://zenodo.org/record/6778336).
The RDF data is directly saved in a graph. Subsequently, the graph is serialized and saved to a TTL and an RDF file. Furthermore, SPARQL queries may be performed for consistency checks.
Hence, all in all, this script is used to create a knowledge graph using vocabulary of several ontologies and tensile test data originating from real tests performed on an S355 steel in accordance with the corresponding test standard [ISO 6892-1:2019-11](https://www.beuth.de/de/norm/iso-6892-1/316885984).

# Installing and Importing relevant Python packages

In [1]:
%%capture
%pip install --upgrade pip
# Installing relevant python package rdflib
%pip install rdflib

In [2]:
# Import of relevant package parts
from rdflib import Graph, Literal, URIRef, BNode, Namespace, DC
from rdflib.term import Identifier
from rdflib.collection import Collection
from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
import rdflib.plugins.sparql.update
import pandas as pd
import io
import urllib.parse

# Definition of helper methods

In [3]:
# Definition of helper methods / functions - do not necessarily have to be used.

# Helper method 'add' used to write triples to an RDF graph
def add(s,p,o):

    # in this case p is "ObjectProperty"
    if o.find('http://')==0 or o.find('https://')==0:
        g.add( (URIRef(s), URIRef(p), URIRef(o)) )

    # in this case p is "DatatypeProperty"
    else:
        # if we can parse o as Float, just set the datatype
        try:
            g.add( (URIRef(s), URIRef(p), Literal(float(o), datatype=XSD.float) ))
        except:
            g.add( (URIRef(s), URIRef(p), Literal(o) ))


# Global counter possibly usable for the creation of arbitrary instance names (numbers)
instanceCounter = 0
def nextInstanceNum():
    global instanceCounter
    instanceCounter = instanceCounter + 1
    return str(instanceCounter)

# Read in tensile test data

In [8]:
# Read in data that is supposed to be mapped to the concepts of the ontology (TTO).
# The data is also findable as a dataset on tensile tests that is published freely available on Zenodo: https://zenodo.org/record/6778336
# Reading in data from URL

# link_data = "https://raw.githubusercontent.com/materialdigital/tensile-test-ontology/main/data/S355_data.csv"
link_data = "data/S355_data.csv"
data_S333_read_in = pd.read_csv(link_data, sep=';')

# Graph creation using RDFlib

In [ ]:
# Creating the RDF graph 'g'
g = Graph()

In [ ]:
# Definition of IRI prefixes (namespaces)
base = Namespace("https://w3id.org/pmd/tto/")
g.bind("base", base)
pmdco = Namespace("https://w3id.org/pmd/co/")
g.bind("pmdco", pmdco)
unit = Namespace("http://qudt.org/vocab/unit/")
g.bind("unit", unit)
prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)
raw_data = Namespace("https://raw.githubusercontent.com/materialdigital/application-ontologies/d8ab894df5d56d3362cedb18c2b482da0836e959/tensile_test_ontology_TTO/data/primary_data/")
g.bind("raw_data", raw_data)
csvw = Namespace("http://www.w3.org/ns/csvw#")
g.bind("csvw", csvw)
datacite = Namespace("http://purl.org/spar/datacite/")
g.bind("datacite", datacite)
obo = Namespace("http://purl.obolibrary.org/obo/")
g.bind("obo", obo)
dct = Namespace("http://purl.org/dc/elements/1.1/")
g.bind("dct", dct)

# A-Box namespace (namespace of instances)
prefix = Namespace("https://w3id.org/pmd/demodata/tensiletest_S355/")
g.bind("prefix", prefix)

# Turtle style abbreviation for RDF.type
a = RDF.type

# Create ontology and import TTO as basis
onto = URIRef(prefix)
g.add((onto, a, OWL.Ontology))
g.add((onto, OWL.imports, URIRef(base)))

# Add creators:
markus = URIRef("https://orcid.org/0000-0002-7094-5371")
bernd = URIRef("https://orcid.org/0000-0002-3717-7104")
joerg = URIRef("https://orcid.org/0000-0001-7192-7143")
philipp = URIRef("https://orcid.org/0000-0003-4971-3645")

g.add((onto, DC.creator, markus))
g.add((markus, RDFS.label, Literal("Markus Schilling", datatype=XSD.string)))
g.add((onto, DC.creator, bernd))
g.add((bernd, RDFS.label, Literal("Bernd Bayerlein", datatype=XSD.string)))
g.add((onto, DC.creator, joerg))
g.add((joerg, RDFS.label, Literal("Jörg Waitelonis", datatype=XSD.string)))
g.add((onto, DC.creator, philipp))
g.add((philipp, RDFS.label, Literal("Philipp von Hartrott", datatype=XSD.string)))

g.add((onto, DC.title, Literal("Tensile Test Ontology (TTO) data mapping example", datatype=XSD.string)))
g.add((onto, OWL.versionInfo, Literal("3.0.0", datatype=XSD.string)))

# Load data from read in above.
data = data_S333_read_in

# Creation of QUDT unit instances as instances of iao:measurement unit label
# mesurement unit label has IRI: obo.IAO_0000003 (see below for more details)
g.add((unit.MilliM, a, obo.IAO_0000003))
g.add((unit.SEC, a, obo.IAO_0000003))
g.add((unit.KiloN, a, obo.IAO_0000003))
g.add((unit.PERCENT, a, obo.IAO_0000003))
g.add((unit.MilliM2, a, obo.IAO_0000003))
g.add((unit.MegaPa, a, obo.IAO_0000003))
g.add((unit.GigaPa, a, obo.IAO_0000003))
g.add((unit.DEG_C, a, obo.IAO_0000003))

# Creation of triples
# Iterate over the full data table
for idx, row in data.iterrows():
  # Definition of tokens in accordance with columns of original csv file
  process_id                          = row["process"]
  testpiece_id                        = row["testpiece"]
  a1                                  = row["d1 / a1"]
  a2                                  = row["d2 / a2"]
  a3                                  = row["d3 / a3"]
  b1                                  = row["b1"]
  b2                                  = row["b2"]
  b3                                  = row["b3"]
  s1                                  = row["s1"]
  s2                                  = row["s2"]
  s3                                  = row["s3"]
  S0                                  = row["S0"]
  Rp02                                = row["Rp0,2"]
  ReH                                 = row["ReH"]
  Fm                                  = row["Fm"]
  Rm                                  = row["Rm"]
  E                                   = row["E"]
  L0                                  = row["L0"]
  Lu                                  = row["Lu"]
  A                                   = row["A"]
  au                                  = row["au"]
  bu                                  = row["bu"]
  Su                                  = row["Su"]
  Z                                   = row["Z"]
  operator                            = row["Operator"]
  date                                = row["date"]
  materialDescription                 = row["Werkstoff"]
  specimenShape                       = row["Probenform"]
  note                                = row["Note"]
  environmentalTemperature            = row["Umgebungstemperatur"]
  machineName                         = row["Prüfmaschinen-Name"]
  machineManufacturer                 = row["Prüfmaschinenhersteller"]
  machineType                         = row["Prüfmaschinen-Typ"]
  machineSerialNumber                 = row["Prüfmaschinen-Seriennummer"]
  machineStandard                     = row["Prüfmaschinen-Standard"]
  extensometerName                    = row["Extensometer"]
  extensometerSerialNumber            = row["Extensometer-Seriennummer"]
  extensometerStandard                = row["Extensometer-Standard"]
  micrometerGaugeManufacturer         = row["Mikrometerschraube-Hersteller"]
  caliperManufacturer                 = row["Messschieber-Hersteller"]
  caliperSerialNumber                 = row["Messschieber-Seriennummer"]
  strainRate                          = row["Dehngeschwindigkeit"]
  transitionPoint                     = row["Umschaltpunkt"]
  extensometerGaugeLength             = row["Extensometer-Nenngerätemesslänger"]
  loadCellMaxForce                    = row["Kraftmessdose-Kraftbereich"]
  project                             = row["Project"]
  standard                            = row["Norm"]
  rollingDirection                    = row["Walzrichtung"]
  location                            = row["Ort"]
  institute                           = row["Institut"]
  address                             = row["Adresse"]
 
  # Creation of triples for all instances (considering all connections between them) based on a bibliographic approach (RDFlib). There is about one "block" per instance.
  experimentIRI = URIRef(prefix + process_id)
  g.add((experimentIRI, a, obo.IAO_0020000))
  g.add((experimentIRI, RDFS.label, Literal(process_id, datatype=XSD.string)))  

  processIRI = URIRef(experimentIRI + "_process")
  g.add((processIRI, a, obo.OBI_0000011)) # Planned process
  g.add((processIRI, a, base.TensileTest)) # Tensile Test
  g.add((experimentIRI, obo.IAO_0000219, processIRI)) # Experiment ID denotes Process (Tensile Test)

  # Creation Process Plan and Process Plan Specification
  processPlanIRI = URIRef(experimentIRI + "_process_plan")
  g.add((processPlanIRI, a, obo.OBI_0000260)) # obi:Plan
  g.add((processIRI, obo.BFO_0000055, processPlanIRI)) # (Tensile Test) Process bfo:realizes the process plan

  processPlanSpecificationIRI = URIRef(experimentIRI + "_process_plan_specification")
  g.add((processPlanSpecificationIRI, a, obo.OBI_0000104)) # obi:Plan Specification
  g.add((processPlanIRI, obo.RO_0000059, processPlanSpecificationIRI)) # The process plan ro:concretizes a process plan specification


  # Advanced process description
  # In this section, the process of tensile testing is disassembled into individual parts, allowing a more precise process and subprocess description.

  # No. 1
  preMeasuringIRI = URIRef(experimentIRI + "_process_preMeasuring")
  g.add((preMeasuringIRI, a, obo.OBI_0000070)) # Assay (subclass of Planned process)
  g.add((preMeasuringIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((processIRI, obo.RO_0002224, preMeasuringIRI)) # (entire) process starts with
  
  # No. 2
  mountingIRI = URIRef(experimentIRI + "_process_mounting")
  g.add((mountingIRI, a, obo.OBI_0000011)) # Planned process
  g.add((mountingIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((mountingIRI, obo.RO_0002090, preMeasuringIRI)) # Mounting process immediately precedes preMeasuring process

  # No. 3
  gripsAlignmentIRI = URIRef(experimentIRI + "_process_gripsAlignment")
  g.add((gripsAlignmentIRI, a, obo.OBI_0000011)) # Planned process
  g.add((gripsAlignmentIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((gripsAlignmentIRI, obo.RO_0002090, mountingIRI)) # Grips Alignment process immediately precedes Mounting process

  # No. 4
  tensileTestISOIRI = URIRef(experimentIRI + "_process_tensileTestISO")
  g.add((tensileTestISOIRI, a, obo.OBI_0000011)) # Planned process
  g.add((tensileTestISOIRI, a, base.TensileTest)) # ISO Tensile testing refers to tensile testing
  g.add((tensileTestISOIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((tensileTestISOIRI, obo.RO_0002090, gripsAlignmentIRI)) # ISO Tensile Test process immediately precedes Grips Alignment process

  # No. 5
  demountingIRI = URIRef(experimentIRI + "_process_demounting")
  g.add((demountingIRI, a, obo.OBI_0000011)) # Planned process
  g.add((demountingIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((demountingIRI, obo.RO_0002090, tensileTestISOIRI)) # Demounting process immediately precedes ISO Tensile Test process

  # No. 6
  postMeasuringIRI = URIRef(experimentIRI + "_process_postMeasuring")
  g.add((postMeasuringIRI, a, obo.OBI_0000070)) # Assay (subclass of Planned process)
  g.add((postMeasuringIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((postMeasuringIRI, obo.RO_0002090, demountingIRI)) # Post Measuring process immediately precedes Demounting process

  # No. 7
  analysisIRI = URIRef(experimentIRI + "_process_analysis")
  g.add((analysisIRI, a, pmdco.ComputingProcess)) # Process is also a Computing process which is a subclass of planned process in acc. with new version of PMDco 3.0 datatransformation module
  # g.add((analysisIRI, a, obo.OBI_0000011)) # Planned process
  g.add((analysisIRI, obo.RO_0002012, processIRI)) # Occurrent part of (entire) process
  g.add((analysisIRI, obo.RO_0002090, postMeasuringIRI)) # Analysis process immediately precedes Post Measuring process

  
  # Object / test piece description
  # Test piece as input prior to tensile test
  testpieceIRI = URIRef(prefix  + f"testpiece/{testpiece_id}")
  g.add((testpieceIRI, a, pmdco.TestPiece))
  testpieceID_IRI = URIRef(prefix + f"testpiece_name/{testpiece_id}")
  g.add((testpieceID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceID_IRI, dct.identifier, Literal(testpiece_id, datatype=XSD.string)))
  g.add((testpieceID_IRI, obo.IAO_0000219, testpieceIRI)) # testpiece ID iao:denotes testpiece

  # Test piece(s) as output after tensile test (typically, 2 fractured parts will be formed)
  # Test piece after test 1
  testpieceAfterTest1IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_1")
  g.add((testpieceAfterTest1IRI, a, pmdco.TestPiece))
  testpieceAfterTest1ID_IRI = URIRef(testpieceID_IRI + "_afterTest_1")
  g.add((testpieceAfterTest1ID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceAfterTest1ID_IRI, dct.identifier, Literal(testpiece_id + "_afterTest_1", datatype=XSD.string)))
  g.add((testpieceAfterTest1ID_IRI, obo.IAO_0000219, testpieceAfterTest1IRI)) # testpiece after test 1 ID iao:denotes testpiece after test 1

  # Test piece after test 2
  testpieceAfterTest2IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_2")
  g.add((testpieceAfterTest2IRI, a, pmdco.TestPiece))
  testpieceAfterTest2ID_IRI = URIRef(testpieceID_IRI + "_afterTest_2")
  g.add((testpieceAfterTest2ID_IRI, a, obo.IAO_0020000)) # IAO:identifier
  g.add((testpieceAfterTest2ID_IRI, dct.identifier, Literal(testpiece_id + "_afterTest_2", datatype=XSD.string)))
  g.add((testpieceAfterTest2ID_IRI, obo.IAO_0000219, testpieceAfterTest2IRI)) # testpiece after test 2 ID iao:denotes testpiece after test 2

  # Process - Test Piece Relations
  g.add((processIRI, obo.OBI_0000293, testpieceIRI)) # Process has specified input testpiece
  g.add((processIRI, obo.OBI_0000299, testpieceAfterTest1IRI)) # Process has specified output testpiece after test 1
  g.add((processIRI, obo.OBI_0000299, testpieceAfterTest2IRI)) # Process has specified output testpiece after test 2
  g.add((testpieceAfterTest1IRI, obo.RO_0001000, testpieceIRI)) # Testpiece after test 1 derives from Testpiece
  g.add((testpieceAfterTest2IRI, obo.RO_0001000, testpieceIRI)) # Testpiece after test 2 derives from Testpiece


  # The next section contains some material / test piece information.
  materialDescriptionIRI = URIRef(experimentIRI + "_materialDescription")
  g.add((materialDescriptionIRI, a, pmdco.materialDesignation))
  g.add((materialDescriptionIRI, a, pmdco.Metadata))
  g.add((materialDescriptionIRI, pmdco.value, Literal(materialDescription, datatype=XSD.string)))
  g.add((testpieceIRI, pmdco.characteristic, materialDescriptionIRI))

  specimenShapeIRI = URIRef(experimentIRI + "_specimenShape")
  g.add((specimenShapeIRI, a, base.GeometryShape))
  g.add((specimenShapeIRI, a, pmdco.Metadata))
  g.add((specimenShapeIRI, pmdco.value, Literal(specimenShape, datatype=XSD.string)))
  g.add((testpieceIRI, pmdco.characteristic, specimenShapeIRI))

  rollingDirectionIRI = URIRef(experimentIRI + "_rollingDirection")
  g.add((rollingDirectionIRI, a, pmdco.MaterialRelated))
  g.add((rollingDirectionIRI, a, pmdco.Metadata))
  g.add((rollingDirectionIRI, pmdco.value, Literal(rollingDirection, datatype=XSD.string)))
  g.add((testpieceIRI, pmdco.characteristic, rollingDirectionIRI))

  # The next section contains some primary data.

  # Raw data / dataset
  datasetIRI = URIRef(prefix + "resource/" + testpiece_id + "_dataset")
  g.add((datasetIRI, a, pmdco.PrimaryData))
  g.add((datasetIRI, a, pmdco.Measurement))
  g.add((datasetIRI, a, pmdco.ValueObject))
  g.add((processIRI, pmdco.output, datasetIRI))

  raw_dataIRI = URIRef(prefix + "resource/" + testpiece_id + "_csv")
  g.add((raw_dataIRI, a, csvw.Table))
  g.add((raw_dataIRI, a, pmdco.Dataset))
  g.add((raw_dataIRI, DC.title, Literal(f"process/{process_id}" + f"testpiece_id/{testpiece_id}", datatype=XSD.string)))
  g.add((raw_dataIRI, csvw.url, Literal(raw_data + f"{testpiece_id}" + ".csv", datatype=XSD.string)))
  g.add((raw_dataIRI, DC.format, Literal("text/csv", datatype=XSD.string)))
  g.add((datasetIRI, pmdco.resource, raw_dataIRI))

  schemaIRI = URIRef(prefix + testpiece_id + "_schema")
  g.add((schemaIRI, a, csvw.Schema))
  g.add((raw_dataIRI, csvw.tableSchema, schemaIRI))

  BNode1 = BNode()
  BNode2 = BNode()
  BNode3 = BNode()
  BNode4 = BNode()

  column1 = BNode()
  g.add((column1, a, csvw.Column))
  g.add((column1, a, pmdco.Duration))
  g.add((column1, pmdco.unit, unit.SEC))
  g.add((column1, csvw.name, Literal("Time", datatype=XSD.string)))
  g.add((column1, DC.title, Literal("Time", datatype=XSD.string)))
  g.add((column1, csvw.datatype, Literal(1, datatype=XSD.float)))
  g.add((column1, csvw.headerCount, Literal(2, datatype=XSD.float)))
  # How to include a header count or skip the (second) header line???

  column2 = BNode()
  g.add((column2, a, csvw.Column))
  g.add((column2, a, base.CrossheadSeparation))
  g.add((column2, pmdco.unit, unit.MilliM))
  g.add((column2, csvw.name, Literal("Crosshead travel", datatype=XSD.string)))
  g.add((column2, DC.title, Literal("Crosshead Separation", datatype=XSD.string)))
  g.add((column2, csvw.datatype, Literal(2, datatype=XSD.float)))
  g.add((column2, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column3 = BNode()
  g.add((column3, a, csvw.Column))
  g.add((column3, a, pmdco.Force))
  g.add((column3, pmdco.unit, unit.KiloN))
  g.add((column3, csvw.name, Literal("Force", datatype=XSD.string)))
  g.add((column3, DC.title, Literal("Force", datatype=XSD.string)))
  g.add((column3, csvw.datatype, Literal(3, datatype=XSD.float)))
  g.add((column3, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column4 = BNode()
  g.add((column4, a, csvw.Column))
  g.add((column4, a, base.PercentageExtension))
  g.add((column4, pmdco.unit, unit.PERCENT))
  g.add((column4, csvw.name, Literal("Strain", datatype=XSD.string)))
  g.add((column4, DC.title, Literal("Percentage Extension", datatype=XSD.string)))
  g.add((column4, csvw.datatype, Literal(4, datatype=XSD.float)))
  g.add((column4, csvw.headerCount, Literal(2, datatype=XSD.float)))

  g.add((schemaIRI, csvw.column, BNode1))
  g.add((BNode1, RDF.first, column1))
  g.add((BNode1, RDF.rest, BNode2))
  g.add((BNode2, RDF.first, column2))
  g.add((BNode2, RDF.rest, BNode3))
  g.add((BNode3, RDF.first, column3))
  g.add((BNode3, RDF.rest, BNode4))
  g.add((BNode4, RDF.first, column4))
  g.add((BNode4, RDF.rest, RDF.nil))

  # Rest of primary data.
  # a1
  a1IRI_value_specification = URIRef(experimentIRI + "_thickness_a1_value_specification")
  g.add((a1IRI_value_specification, a, obo.OBI_0001931)) # a1 measurement value specification is a obi:scalar value specification
  g.add((a1IRI_value_specification, obo.OBI_0001937, Literal(a1, datatype=XSD.float))) # 
  g.add((a1IRI_value_specification, obo.IAO_0000039, unit.MilliM)) # a1 measurement value specification iao:has measurement unit label unit:MilliM (QUDT)
  g.add((a1IRI_value_specification, obo.IAO_0000136, testpieceIRI)) # a1 measurement value specification iao:is about Testpiece

  a1IRI = URIRef(experimentIRI + "_thickness_a1_scalar_measurement_datum")
  g.add((a1IRI, a, obo.IAO_0000032)) # a1 is a iao:scalar measurement datum
  g.add((a1IRI, obo.OBI_0001938, a1IRI_value_specification)) # a1 scalar measurement datum obi:has specified numeric value a1 measurement value specification
  g.add((processIRI, obo.OBI_0000299, a1IRI)) # Process has specified output a1
  g.add((preMeasuringIRI, obo.OBI_0000299, a1IRI)) # preMeasuring process has specified output a1
  g.add((a1IRI, obo.IAO_0000136, testpieceIRI)) # a1 scalar measurement datum iao:is about testpiece

  a1IRI_quality = URIRef(experimentIRI + "_thickness_a1")
  g.add((a1IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a1IRI_quality, obo.IAO_0000417, a1IRI)) # a1 quality iao:is quality measured as a1
  g.add((testpieceIRI, obo.RO_0000086, a1IRI_quality)) # Testpiece ro:has quality a1 quality

  # a2
  a2IRI_value_specification = URIRef(experimentIRI + "_thickness_a2_value_specification")
  g.add((a2IRI_value_specification, a, obo.OBI_0001931))
  g.add((a2IRI_value_specification, obo.OBI_0001937, Literal(a2, datatype=XSD.float)))
  g.add((a2IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((a2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  a2IRI = URIRef(experimentIRI + "_thickness_a2_scalar_measurement_datum")
  g.add((a2IRI, a, obo.IAO_0000032))
  g.add((a2IRI, obo.OBI_0001938, a2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, a2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, a2IRI)) # preMeasuring process has specified output a2
  g.add((a2IRI, obo.IAO_0000136, testpieceIRI))

  a2IRI_quality = URIRef(experimentIRI + "_thickness_a2")
  g.add((a2IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a2IRI_quality, obo.IAO_0000417, a2IRI))
  g.add((testpieceIRI, obo.RO_0000086, a2IRI_quality))

  # a3
  a3IRI_value_specification = URIRef(experimentIRI + "_thickness_a3_value_specification")
  g.add((a3IRI_value_specification, a, obo.OBI_0001931))
  g.add((a3IRI_value_specification, obo.OBI_0001937, Literal(a3, datatype=XSD.float)))
  g.add((a3IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((a3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  a3IRI = URIRef(experimentIRI + "_thickness_a3_scalar_measurement_datum")
  g.add((a3IRI, a, obo.IAO_0000032))
  g.add((a3IRI, obo.OBI_0001938, a3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, a3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, a3IRI)) # preMeasuring process has specified output a3
  g.add((a3IRI, obo.IAO_0000136, testpieceIRI))

  a3IRI_quality = URIRef(experimentIRI + "_thickness_a3")
  g.add((a3IRI_quality, a, base.OriginalThickness)) # Original Thickness used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((a3IRI_quality, obo.IAO_0000417, a3IRI))
  g.add((testpieceIRI, obo.RO_0000086, a3IRI_quality))

  # b1
  b1IRI_value_specification = URIRef(experimentIRI + "_width_b1_value_specification")
  g.add((b1IRI_value_specification, a, obo.OBI_0001931))
  g.add((b1IRI_value_specification, obo.OBI_0001937, Literal(b1, datatype=XSD.float)))
  g.add((b1IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b1IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b1IRI = URIRef(experimentIRI + "_width_b1_scalar_measurement_datum")
  g.add((b1IRI, a, obo.IAO_0000032))
  g.add((b1IRI, obo.OBI_0001938, b1IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b1IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b1IRI)) # preMeasuring process has specified output b1
  g.add((b1IRI, obo.IAO_0000136, testpieceIRI))

  b1IRI_quality = URIRef(experimentIRI + "_width_b1")
  g.add((b1IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b1IRI_quality, obo.IAO_0000417, b1IRI))
  g.add((testpieceIRI, obo.RO_0000086, b1IRI_quality))

  # b2
  b2IRI_value_specification = URIRef(experimentIRI + "_width_b2_value_specification")
  g.add((b2IRI_value_specification, a, obo.OBI_0001931))
  g.add((b2IRI_value_specification, obo.OBI_0001937, Literal(b2, datatype=XSD.float)))
  g.add((b2IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b2IRI = URIRef(experimentIRI + "_width_b2_scalar_measurement_datum")
  g.add((b2IRI, a, obo.IAO_0000032))
  g.add((b2IRI, obo.OBI_0001938, b2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b2IRI)) # preMeasuring process has specified output b2
  g.add((b2IRI, obo.IAO_0000136, testpieceIRI))

  b2IRI_quality = URIRef(experimentIRI + "_width_b2")
  g.add((b2IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b2IRI_quality, obo.IAO_0000417, b2IRI))
  g.add((testpieceIRI, obo.RO_0000086, b2IRI_quality))

  # b3
  b3IRI_value_specification = URIRef(experimentIRI + "_width_b3_value_specification")
  g.add((b3IRI_value_specification, a, obo.OBI_0001931))
  g.add((b3IRI_value_specification, obo.OBI_0001937, Literal(b3, datatype=XSD.float)))
  g.add((b3IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((b3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  b3IRI = URIRef(experimentIRI + "_width_b3_scalar_measurement_datum")
  g.add((b3IRI, a, obo.IAO_0000032))
  g.add((b3IRI, obo.OBI_0001938, b3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, b3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, b3IRI)) # preMeasuring process has specified output b3
  g.add((b3IRI, obo.IAO_0000136, testpieceIRI))

  b3IRI_quality = URIRef(experimentIRI + "_width_b3")
  g.add((b3IRI_quality, a, base.OriginalWidth)) # Original Width used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((b3IRI_quality, obo.IAO_0000417, b3IRI))
  g.add((testpieceIRI, obo.RO_0000086, b3IRI_quality))

  # s1
  s1IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s1_value_specification")
  g.add((s1IRI_value_specification, a, obo.OBI_0001931))
  g.add((s1IRI_value_specification, obo.OBI_0001937, Literal(s1, datatype=XSD.float)))
  g.add((s1IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s1IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s1IRI = URIRef(experimentIRI + "_crossSectionArea_s1_scalar_measurement_datum")
  g.add((s1IRI, a, obo.IAO_0000032))
  g.add((s1IRI, obo.OBI_0001938, s1IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s1IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s1IRI)) # preMeasuring process has specified output s1
  g.add((s1IRI, obo.IAO_0000136, testpieceIRI))

  s1IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s1")
  g.add((s1IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s1IRI_quality, obo.IAO_0000417, s1IRI))
  g.add((testpieceIRI, obo.RO_0000086, s1IRI_quality))

  # s2
  s2IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s2_value_specification")
  g.add((s2IRI_value_specification, a, obo.OBI_0001931))
  g.add((s2IRI_value_specification, obo.OBI_0001937, Literal(s2, datatype=XSD.float)))
  g.add((s2IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s2IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s2IRI = URIRef(experimentIRI + "_crossSectionArea_s2_scalar_measurement_datum")
  g.add((s2IRI, a, obo.IAO_0000032))
  g.add((s2IRI, obo.OBI_0001938, s2IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s2IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s2IRI)) # preMeasuring process has specified output s2
  g.add((s2IRI, obo.IAO_0000136, testpieceIRI))

  s2IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s2")
  g.add((s2IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s2IRI_quality, obo.IAO_0000417, s2IRI))
  g.add((testpieceIRI, obo.RO_0000086, s2IRI_quality))

  # s3
  s3IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s3_value_specification")
  g.add((s3IRI_value_specification, a, obo.OBI_0001931))
  g.add((s3IRI_value_specification, obo.OBI_0001937, Literal(s3, datatype=XSD.float)))
  g.add((s3IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s3IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s3IRI = URIRef(experimentIRI + "_crossSectionArea_s3_scalar_measurement_datum")
  g.add((s3IRI, a, obo.IAO_0000032))
  g.add((s3IRI, obo.OBI_0001938, s3IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s3IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s3IRI)) # preMeasuring process has specified output s3
  g.add((s3IRI, obo.IAO_0000136, testpieceIRI))

  s3IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s3")
  g.add((s3IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s3IRI_quality, obo.IAO_0000417, s3IRI))
  g.add((testpieceIRI, obo.RO_0000086, s3IRI_quality))

  # S0
  s0IRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_s0_value_specification")
  g.add((s0IRI_value_specification, a, obo.OBI_0001931))
  g.add((s0IRI_value_specification, obo.OBI_0001937, Literal(S0, datatype=XSD.float)))
  g.add((s0IRI_value_specification, obo.IAO_0000039, unit.MilliM2))
  g.add((s0IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  s0IRI = URIRef(experimentIRI + "_crossSectionArea_s0_scalar_measurement_datum")
  g.add((s0IRI, a, obo.IAO_0000032))
  g.add((s0IRI, obo.OBI_0001938, s0IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, s0IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, s0IRI)) # preMeasuring process has specified output S0
  g.add((s0IRI, obo.IAO_0000136, testpieceIRI))

  s0IRI_quality = URIRef(experimentIRI + "_crossSectionArea_s0")
  g.add((s0IRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((s0IRI_quality, obo.IAO_0000417, s0IRI))
  g.add((testpieceIRI, obo.RO_0000086, s0IRI_quality))

  # L0
  L0IRI_value_specification = URIRef(experimentIRI + "_originalGaugeLength_L0_value_specification")
  g.add((L0IRI_value_specification, a, obo.OBI_0001931))
  g.add((L0IRI_value_specification, obo.OBI_0001937, Literal(L0, datatype=XSD.float)))
  g.add((L0IRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((L0IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  L0IRI = URIRef(experimentIRI + "_originalGaugeLength_L0_scalar_measurement_datum")
  g.add((L0IRI, a, obo.IAO_0000032))
  g.add((L0IRI, obo.OBI_0001938, L0IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, L0IRI))
  g.add((preMeasuringIRI, obo.OBI_0000299, L0IRI)) # preMeasuring process has specified output L0
  g.add((L0IRI, obo.IAO_0000136, testpieceIRI))

  L0IRI_quality = URIRef(experimentIRI + "_originalGaugeLength_L0")
  g.add((L0IRI_quality, a, base.OriginalGaugeLength)) # Original Gauge Length used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((L0IRI_quality, obo.IAO_0000417, L0IRI))
  g.add((testpieceIRI, obo.RO_0000086, L0IRI_quality))

  # Lu
  LuIRI_value_specification = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu_value_specification")
  g.add((LuIRI_value_specification, a, obo.OBI_0001931))
  g.add((LuIRI_value_specification, obo.OBI_0001937, Literal(Lu, datatype=XSD.float)))
  g.add((LuIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((LuIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  LuIRI = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu_scalar_measurement_datum")
  g.add((LuIRI, a, obo.IAO_0000032))
  g.add((LuIRI, obo.OBI_0001938, LuIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, LuIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, LuIRI)) # postMeasuring process has specified output Lu
  g.add((LuIRI, obo.IAO_0000136, testpieceIRI))

  LuIRI_quality = URIRef(experimentIRI + "_finalGaugeLengthAfterFracture_Lu")
  g.add((LuIRI_quality, a, base.FinalGaugeLengthAfterFracture)) # Final Gauge Length After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((LuIRI_quality, obo.IAO_0000417, LuIRI))
  g.add((testpieceIRI, obo.RO_0000086, LuIRI_quality))

  # au
  auIRI_value_specification = URIRef(experimentIRI + "_thicknessAfterFracture_au_value_specification")
  g.add((auIRI_value_specification, a, obo.OBI_0001931))
  g.add((auIRI_value_specification, obo.OBI_0001937, Literal(au, datatype=XSD.float)))
  g.add((auIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((auIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  auIRI = URIRef(experimentIRI + "_thicknessAfterFracture_au_scalar_measurement_datum")
  g.add((auIRI, a, obo.IAO_0000032))
  g.add((auIRI, obo.OBI_0001938, auIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, auIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, auIRI)) # postMeasuring process has specified output au
  g.add((auIRI, obo.IAO_0000136, testpieceIRI))

  auIRI_quality = URIRef(experimentIRI + "_thicknessAfterFracture_au")
  g.add((auIRI_quality, a, base.ThicknessAfterFracture)) # Thickness After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((auIRI_quality, obo.IAO_0000417, auIRI))
  g.add((testpieceIRI, obo.RO_0000086, auIRI_quality))

  # bu
  buIRI_value_specification = URIRef(experimentIRI + "_widthAfterFracture_bu_value_specification")
  g.add((buIRI_value_specification, a, obo.OBI_0001931))
  g.add((buIRI_value_specification, obo.OBI_0001937, Literal(bu, datatype=XSD.float)))
  g.add((buIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((buIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  buIRI = URIRef(experimentIRI + "_widthAfterFracture_bu_scalar_measurement_datum")
  g.add((buIRI, a, obo.IAO_0000032))
  g.add((buIRI, obo.OBI_0001938, buIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, buIRI))
  g.add((postMeasuringIRI, obo.OBI_0000299, buIRI)) # postMeasuring process has specified output bu
  g.add((buIRI, obo.IAO_0000136, testpieceIRI))

  buIRI_quality = URIRef(experimentIRI + "_widthAfterFracture_bu")
  g.add((buIRI_quality, a, base.WidthAfterFracture)) # Width After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((buIRI_quality, obo.IAO_0000417, buIRI))
  g.add((testpieceIRI, obo.RO_0000086, buIRI_quality))

  # Su
  SuIRI_value_specification = URIRef(experimentIRI + "_crossSectionArea_Su_value_specification")
  g.add((SuIRI_value_specification, a, obo.OBI_0001931))
  g.add((SuIRI_value_specification, obo.OBI_0001937, Literal(Su, datatype=XSD.float)))
  g.add((SuIRI_value_specification, obo.IAO_0000039, unit.MilliM))
  g.add((SuIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  SuIRI = URIRef(experimentIRI + "_crossSectionArea_Su_scalar_measurement_datum")
  g.add((SuIRI, a, obo.IAO_0000032))
  g.add((SuIRI, obo.OBI_0001938, SuIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, SuIRI))
  g.add((analysisIRI, obo.OBI_0000299, SuIRI)) # Analysis process has specified output Su, since Su is calculated within this process
  g.add((SuIRI, obo.IAO_0000136, testpieceIRI))

  SuIRI_quality = URIRef(experimentIRI + "_crossSectionArea_Su")
  g.add((SuIRI_quality, a, pmdco.CrossSectionArea)) # Cross Section Area (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((SuIRI_quality, obo.IAO_0000417, SuIRI))
  g.add((testpieceIRI, obo.RO_0000086, SuIRI_quality))

  # The next section contains some secondary data.

  # Rp02
  Rp02IRI_value_specification = URIRef(experimentIRI + "_proofStrength_Rp02_value_specification")
  g.add((Rp02IRI_value_specification, a, obo.OBI_0001931))
  g.add((Rp02IRI_value_specification, obo.OBI_0001937, Literal(Rp02, datatype=XSD.float)))
  g.add((Rp02IRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((Rp02IRI_value_specification, obo.IAO_0000136, testpieceIRI))

  Rp02IRI = URIRef(experimentIRI + "_proofStrength_Rp02_scalar_measurement_datum")
  g.add((Rp02IRI, a, obo.IAO_0000032))
  g.add((Rp02IRI, obo.OBI_0001938, Rp02IRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, Rp02IRI))
  g.add((analysisIRI, obo.OBI_0000299, Rp02IRI)) # Analysis process has specified output Rp02, since it is calculated / computed within this process
  g.add((Rp02IRI, obo.IAO_0000136, testpieceIRI))

  Rp02IRI_quality = URIRef(experimentIRI + "_proofStrength_Rp02")
  g.add((Rp02IRI_quality, a, base.Rp02)) # Rp02 used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((Rp02IRI_quality, obo.IAO_0000417, Rp02IRI))
  g.add((testpieceIRI, obo.RO_0000086, Rp02IRI_quality))

  # ReH
  ReHIRI_value_specification = URIRef(experimentIRI + "_upperYieldStrength_ReH_value_specification")
  g.add((ReHIRI_value_specification, a, obo.OBI_0001931))
  g.add((ReHIRI_value_specification, obo.OBI_0001937, Literal(ReH, datatype=XSD.float)))
  g.add((ReHIRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((ReHIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  ReHIRI = URIRef(experimentIRI + "_upperYieldStrength_ReH_scalar_measurement_datum")
  g.add((ReHIRI, a, obo.IAO_0000032))
  g.add((ReHIRI, obo.OBI_0001938, ReHIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, ReHIRI))
  g.add((analysisIRI, obo.OBI_0000299, ReHIRI)) # Analysis process has specified output ReH, since it is calculated / computed within this process
  g.add((ReHIRI, obo.IAO_0000136, testpieceIRI))

  ReHIRI_quality = URIRef(experimentIRI + "_upperYieldStrength_ReH")
  g.add((ReHIRI_quality, a, base.UpperYieldStrength)) # Upper Yield Strength used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((ReHIRI_quality, obo.IAO_0000417, ReHIRI))
  g.add((testpieceIRI, obo.RO_0000086, ReHIRI_quality))

  # Fm
  FmIRI_value_specification = URIRef(experimentIRI + "_maximumForce_Fm_value_specification")
  g.add((FmIRI_value_specification, a, obo.OBI_0001931))
  g.add((FmIRI_value_specification, obo.OBI_0001937, Literal(Fm, datatype=XSD.float)))
  g.add((FmIRI_value_specification, obo.IAO_0000039, unit.kiloN))
  g.add((FmIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  FmIRI = URIRef(experimentIRI + "_maximumForce_Fm_scalar_measurement_datum")
  g.add((FmIRI, a, obo.IAO_0000032))
  g.add((FmIRI, obo.OBI_0001938, FmIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, FmIRI))
  g.add((analysisIRI, obo.OBI_0000299, FmIRI)) # Analysis process has specified output Fm, since it is calculated / computed within this process
  g.add((FmIRI, obo.IAO_0000136, testpieceIRI))

  FmIRI_quality = URIRef(experimentIRI + "_maximumForce_Fm")
  g.add((FmIRI_quality, a, base.MaximumForce)) # Maximum Force used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((FmIRI_quality, obo.IAO_0000417, FmIRI))
  g.add((testpieceIRI, obo.RO_0000086, FmIRI_quality))

  # Rm
  RmIRI_value_specification = URIRef(experimentIRI + "_tensileStrength_Rm_value_specification")
  g.add((RmIRI_value_specification, a, obo.OBI_0001931))
  g.add((RmIRI_value_specification, obo.OBI_0001937, Literal(Rm, datatype=XSD.float)))
  g.add((RmIRI_value_specification, obo.IAO_0000039, unit.MegaPa))
  g.add((RmIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  RmIRI = URIRef(experimentIRI + "_tensileStrength_Rm_scalar_measurement_datum")
  g.add((RmIRI, a, obo.IAO_0000032))
  g.add((RmIRI, obo.OBI_0001938, RmIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, RmIRI))
  g.add((analysisIRI, obo.OBI_0000299, RmIRI)) # Analysis process has specified output Rm, since it is calculated / computed within this process
  g.add((RmIRI, obo.IAO_0000136, testpieceIRI))

  RmIRI_quality = URIRef(experimentIRI + "_tensileStrength_Rm")
  g.add((RmIRI_quality, a, base.TensileStrength)) # Tensile Strength used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((RmIRI_quality, obo.IAO_0000417, RmIRI))
  g.add((testpieceIRI, obo.RO_0000086, RmIRI_quality))

  # E
  EIRI_value_specification = URIRef(experimentIRI + "_modulusOfElasticity_E_value_specification")
  g.add((EIRI_value_specification, a, obo.OBI_0001931))
  g.add((EIRI_value_specification, obo.OBI_0001937, Literal(E, datatype=XSD.float)))
  g.add((EIRI_value_specification, obo.IAO_0000039, unit.GigaPa))
  g.add((EIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  EIRI = URIRef(experimentIRI + "_modulusOfElasticity_E_scalar_measurement_datum")
  g.add((EIRI, a, obo.IAO_0000032))
  g.add((EIRI, obo.OBI_0001938, EIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, EIRI))
  g.add((analysisIRI, obo.OBI_0000299, EIRI)) # Analysis process has specified output E, since it is calculated / computed within this process
  g.add((EIRI, obo.IAO_0000136, testpieceIRI))

  EIRI_quality = URIRef(experimentIRI + "_modulusOfElasticity_E")
  g.add((EIRI_quality, a, pmdco.ModulusOfElasticity)) # Modulus Of Elasticity (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((EIRI_quality, obo.IAO_0000417, EIRI))
  g.add((testpieceIRI, obo.RO_0000086, EIRI_quality))

  # Slope of the elastic part
  slopeElasticPartIRI_value_specification = URIRef(experimentIRI + "_slopeOfTheElasticPart_me_value_specification")
  g.add((slopeElasticPartIRI_value_specification, a, obo.OBI_0001931))
  g.add((slopeElasticPartIRI_value_specification, obo.OBI_0001937, Literal(E, datatype=XSD.float)))
  g.add((slopeElasticPartIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  slopeElasticPartIRI = URIRef(experimentIRI + "_slopeOfTheElasticPart_me_scalar_measurement_datum")
  g.add((slopeElasticPartIRI, a, obo.IAO_0000032))
  g.add((slopeElasticPartIRI, obo.OBI_0001938, slopeElasticPartIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, slopeElasticPartIRI))
  g.add((analysisIRI, obo.OBI_0000299, slopeElasticPartIRI)) # Analysis process has specified output slope of the elastic part, since it is calculated / computed within this process
  g.add((slopeElasticPartIRI, obo.IAO_0000136, testpieceIRI))

  slopeElasticPartIRI_quality = URIRef(experimentIRI + "_slopeOfTheElasticPart_me")
  g.add((slopeElasticPartIRI_quality, a, base.SlopeOfTheElasticPart)) # Slope Of The Elastic Part used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((slopeElasticPartIRI_quality, obo.IAO_0000417, slopeElasticPartIRI))
  g.add((testpieceIRI, obo.RO_0000086, slopeElasticPartIRI_quality))

  # A
  AIRI_value_specification = URIRef(experimentIRI + "_percentageElongationAfterFracture_A_value_specification")
  g.add((AIRI_value_specification, a, obo.OBI_0001931))
  g.add((AIRI_value_specification, obo.OBI_0001937, Literal(A, datatype=XSD.float)))
  g.add((AIRI_value_specification, obo.IAO_0000039, unit.PERCENT))
  g.add((AIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  AIRI = URIRef(experimentIRI + "_percentageElongationAfterFracture_A_scalar_measurement_datum")
  g.add((AIRI, a, obo.IAO_0000032))
  g.add((AIRI, obo.OBI_0001938, AIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, AIRI))
  g.add((analysisIRI, obo.OBI_0000299, AIRI)) # Analysis process has specified output A, since it is calculated / computed within this process
  g.add((AIRI, obo.IAO_0000136, testpieceIRI))

  AIRI_quality = URIRef(experimentIRI + "_percentageElongationAfterFracture_A")
  g.add((AIRI_quality, a, base.PercentageElongationAfterFracture)) # Percentage Elongation After Fracture used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((AIRI_quality, obo.IAO_0000417, AIRI))
  g.add((testpieceIRI, obo.RO_0000086, AIRI_quality))

  # Z
  ZIRI_value_specification = URIRef(experimentIRI + "_percentageReductionOfArea_Z_value_specification")
  g.add((ZIRI_value_specification, a, obo.OBI_0001931))
  g.add((ZIRI_value_specification, obo.OBI_0001937, Literal(Z, datatype=XSD.float)))
  g.add((ZIRI_value_specification, obo.IAO_0000039, unit.PERCENT))
  g.add((ZIRI_value_specification, obo.IAO_0000136, testpieceIRI))

  ZIRI = URIRef(experimentIRI + "_percentageReductionOfArea_Z_scalar_measurement_datum")
  g.add((ZIRI, a, obo.IAO_0000032))
  g.add((ZIRI, obo.OBI_0001938, ZIRI_value_specification))
  g.add((processIRI, obo.OBI_0000299, ZIRI))
  g.add((analysisIRI, obo.OBI_0000299, ZIRI)) # Analysis process has specified output Z, since it is calculated / computed within this process
  g.add((ZIRI, obo.IAO_0000136, testpieceIRI))

  ZIRI_quality = URIRef(experimentIRI + "_percentageReductionOfArea_Z")
  g.add((ZIRI_quality, a, base.PercentageReductionOfArea)) # Percentage Reduction Of Area used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((ZIRI_quality, obo.IAO_0000417, ZIRI))
  g.add((testpieceIRI, obo.RO_0000086, ZIRI_quality))

  # The next section contains metadata concerning the process itself.

  #####################################################

  # Extensometer Gauge Length
  extensometerGaugeLengthIRI_value_specification = URIRef(experimentIRI + "_extensometerGaugeLength_value_specification")
  g.add((extensometerGaugeLengthIRI_value_specification, a, obo.OBI_0001931))
  g.add((extensometerGaugeLengthIRI_value_specification, obo.OBI_0001937, Literal(extensometerGaugeLength, datatype=XSD.float)))
  g.add((extensometerGaugeLengthIRI_value_specification, obo.IAO_0000039, unit.MilliM))

  extensometerGaugeLengthIRI = URIRef(experimentIRI + "_extensometerGaugeLength_scalar_measurement_datum")
  g.add((extensometerGaugeLengthIRI, a, obo.IAO_0000032))
  g.add((extensometerGaugeLengthIRI, obo.OBI_0001938, extensometerGaugeLengthIRI_value_specification))

  extensometerGaugeLengthIRI_quality = URIRef(experimentIRI + "_extensometerGaugeLength")
  g.add((extensometerGaugeLengthIRI_quality, a, base.ExtensometerGaugeLength)) # Extensometer Gauge Length used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((extensometerGaugeLengthIRI_quality, obo.IAO_0000417, extensometerGaugeLengthIRI))
  g.add((extensometerGaugeLengthIRI_value_specification, obo.OBI_0001927, extensometerGaugeLengthIRI_quality))
  g.add((extensometerIRI, obo.RO_0000086, extensometerGaugeLengthIRI_quality)) # Device Extensometer needs to be defined beforehand

  # Environmental Temperature
  temperatureIRI_value_specification = URIRef(experimentIRI + "_environmentalTemperature_value_specification")
  g.add((temperatureIRI_value_specification, a, obo.OBI_0001931))
  g.add((temperatureIRI_value_specification, obo.OBI_0001937, Literal(environmentalTemperature, datatype=XSD.float)))
  g.add((temperatureIRI_value_specification, obo.IAO_0000039, unit.DEG_C))

  temperatureIRI = URIRef(experimentIRI + "_environmentalTemperature_scalar_measurement_datum")
  g.add((temperatureIRI, a, obo.IAO_0000032))
  g.add((temperatureIRI, obo.OBI_0001938, temperatureIRI_value_specification))

  temperatureIRI_quality = URIRef(experimentIRI + "_environmentalTemperature")
  g.add((temperatureIRI_quality, a, pmdco.EnvironmentalTemperature)) # Environmental Temperature (pmdco) used as subclass of pmdco:quality (instead, pmdco.quality could be used)
  g.add((temperatureIRI_quality, obo.IAO_0000417, temperatureIRI))
  g.add((temperatureIRI_value_specification, obo.OBI_0001927, temperatureIRI_quality))
  # Temperature needs to be put somewhere --> to the process?



  dateIRI = URIRef(experimentIRI + "_date")
  g.add((dateIRI, a, pmdco.Date))
  g.add((dateIRI, a, pmdco.Metadata))
  g.add((dateIRI, pmdco.value, Literal(date)))
  g.add((processIRI, pmdco.characteristic, dateIRI)) 

  strainRateIRI = URIRef(experimentIRI + "_strainRate")
  g.add((strainRateIRI, a, base.StrainRate))
  g.add((strainRateIRI, a, pmdco.Metadata))
  g.add((strainRateIRI, a, pmdco.SetPoint))
  g.add((strainRateIRI, pmdco.value, Literal(strainRate, datatype=XSD.float)))
  g.add((processIRI, pmdco.characteristic, strainRateIRI))

  transitionPointIRI = URIRef(experimentIRI + "_transitionPoint")
  g.add((transitionPointIRI, a, base.TransitionPointTestingRate))
  g.add((transitionPointIRI, a, pmdco.Metadata))
  g.add((transitionPointIRI, a, pmdco.SetPoint))
  g.add((transitionPointIRI, pmdco.value, Literal(transitionPoint, datatype=XSD.float)))
  g.add((transitionPointIRI, pmdco.unit, unit.PERCENT))
  g.add((processIRI, pmdco.characteristic, transitionPointIRI))

  projectIRI = URIRef(experimentIRI + "_project")
  g.add((projectIRI, a, pmdco.Project))
  g.add((projectIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.characteristic, projectIRI))

  project_idIRI = URIRef(experimentIRI + "_project_ID")
  g.add((project_idIRI, a, pmdco.ProjectIdentifier))
  g.add((project_idIRI, a, pmdco.Metadata))
  g.add((project_idIRI, pmdco.value, Literal(project, datatype=XSD.string)))
  g.add((projectIRI, pmdco.characteristic, project_idIRI))

  operatorIRI = URIRef(experimentIRI + "_operator")
  g.add((operatorIRI, a, pmdco.Operator))
  g.add((operatorIRI, a, pmdco.Metadata))
  g.add((operatorIRI, pmdco.value, Literal(operator, datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, operatorIRI))

  standardIRI = URIRef(prefix + "_standard")
  g.add((standardIRI, a, pmdco.Norm))
  g.add((standardIRI, a, pmdco.Metadata))
  g.add((standardIRI, pmdco.value, Literal(standard, datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, standardIRI))

  noteIRI = URIRef(experimentIRI + "_note")
  g.add((noteIRI, a, pmdco.Note))
  g.add((noteIRI, a, pmdco.Metadata))
  g.add((noteIRI, pmdco.value, Literal(note, datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, noteIRI))

  # The next section contains metadata on location, address, institute, etc.
  # 'Static' data such as institution and location get a static URI (the same URI / instance for every process regarded)
  # (there is only one of the explicitely specified institution and location that is referred to, respectively)
  institutionIRI = URIRef(prefix + "_institute_BAM")
  g.add((institutionIRI, a, prov.Organization))
  g.add((institutionIRI, a, pmdco.Metadata))
  g.add((institutionIRI, RDFS.label, Literal(institute, datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, institutionIRI))

  locationIRI = URIRef(prefix + "_location_Berlin")
  g.add((locationIRI, a, prov.Location))
  g.add((locationIRI, a, pmdco.Metadata))
  g.add((locationIRI, RDFS.label, Literal(location, datatype=XSD.string)))
  g.add((institutionIRI, pmdco.characteristic, locationIRI))

  addressIRI = URIRef(prefix + "_institute_BAM_address")
  g.add((addressIRI, a, pmdco.Address))
  g.add((addressIRI, a, pmdco.Metadata))
  g.add((addressIRI, pmdco.value, Literal(address, datatype=XSD.string)))
  g.add((institutionIRI, pmdco.characteristic, addressIRI))

  # The next section contains metadata concerning machines, components (processing nodes) used.
  # 'Static' data such as manufacturers and serial numbers get a static URI (the same URI / instance for every process regarded)
  # Caution: If several machines were used, the machineIRI will have to be changed:
  # Then, it will have to contain the machine designation in the IRI (cf. e.g. institute)
  machineFunctionIRI = URIRef(prefix + "_tensile_testing_machine_function")
  g.add((machineFunctionIRI, a, base.TensileTestingFunction)) # is a tto:tensile testing function
  g.add((machineFunctionIRI, obo.BFO_0000054, processIRI)) # tensile testing function is bfo:realized in (tensile testing) process
  g.add((machineFunctionIRI, obo.BFO_0000054, tensileTestISOIRI)) # tensile testing function is bfo:realized in ISO tensile testing process (subprocess)

  machineIRI = URIRef(prefix + "_tensile_testing_machine")
  g.add((machineIRI, a, base.TensileTestingMachine)) # Machine is a tto:Tensile Testing Machine
  g.add((machineIRI, obo.RO_0000085, machineFunctionIRI)) # Tensile Testing Machine ro:has function Tensile Testing Function
  g.add((machineIRI, obo.RO_0002215, processIRI)) # Tensile Testing Machine is ro:capable of (tensile testing) process (can realize the process)
  g.add((machineIRI, obo.RO_0002215, tensileTestISOIRI)) # Tensile Testing Machine is ro:capable of ISO tensile testing process (subprocess, can realize the process)
  g.add((processIRI, obo.RO_0002608, machineIRI)) # Process (tensile testing) ro:process has causal agent Tensile Testing Machine
  g.add((tensileTestISOIRI, obo.RO_0002608, machineIRI)) # ISO tensile testing process ro:process has causal agent Tensile Testing Machine
  

  machineNameIRI = URIRef(prefix + "_tensile_testing_machine_name")
  g.add((machineNameIRI, a, obo.IAO_0020000)) # is an iao:identifier
  g.add((machineNameIRI, dct.identifier, Literal(machineName, datatype=XSD.string)))
  g.add((machineNameIRI, obo.IAO_0000219, machineIRI)) # Machine Name (identifier) aio:denotes Machine (Tensile Testing Machine)
  

  machineManufacturerIRI = URIRef(prefix + "_machineManufacturer")
  g.add((machineManufacturerIRI, a, pmdco.Manufacturer))
  g.add((machineManufacturerIRI, a, pmdco.Metadata))
  g.add((machineManufacturerIRI, pmdco.value, Literal(machineManufacturer, datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineManufacturerIRI))

  machineTypeIRI = URIRef(prefix + "_machineType_" + f"{machineType}")
  g.add((machineTypeIRI, a, pmdco.NodeType))
  g.add((machineTypeIRI, a, pmdco.Metadata))
  g.add((machineTypeIRI, pmdco.value, Literal(machineType, datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineTypeIRI))

  machineSerialNumberIRI = URIRef(prefix + "_machineSerialNumber_" + machineSerialNumber)
  g.add((machineSerialNumberIRI, a, pmdco.NodeSerialNumber))
  g.add((machineSerialNumberIRI, a, pmdco.Metadata))
  g.add((machineSerialNumberIRI, pmdco.value, Literal(machineSerialNumber, datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineSerialNumberIRI))

  machineStandardIRI = URIRef(prefix + "_machineStandard")
  g.add((machineStandardIRI, a, pmdco.Norm))
  g.add((machineStandardIRI, a, pmdco.Metadata))
  g.add((machineStandardIRI, pmdco.value, Literal(machineStandard, datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineStandardIRI))

  extensometerIRI = URIRef(prefix + "_extensometer")
  g.add((machineIRI, a, pmdco.Extensometer))
  g.add((machineIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.executedBy, extensometerIRI))

  extensometerNameIRI = URIRef(prefix + "_extensometerName")
  g.add((extensometerNameIRI, a, pmdco.NodeName))
  g.add((extensometerNameIRI, a, pmdco.Metadata))
  g.add((extensometerNameIRI, pmdco.value, Literal(extensometerName, datatype=XSD.string)))
  g.add((extensometerIRI, pmdco.characteristic, extensometerNameIRI))

  extensometerSerialNumberIRI = URIRef(prefix + "_extensometerSerialNumber_" + extensometerSerialNumber)
  g.add((extensometerSerialNumberIRI, a, pmdco.NodeSerialNumber))
  g.add((extensometerSerialNumberIRI, a, pmdco.Metadata))
  g.add((extensometerSerialNumberIRI, pmdco.value, Literal(extensometerSerialNumber, datatype=XSD.string)))
  g.add((extensometerIRI, pmdco.characteristic, extensometerSerialNumberIRI))

  extensometerStandardIRI = URIRef(prefix + "_extensometerStandard")
  g.add((extensometerStandardIRI, a, pmdco.Norm))
  g.add((extensometerStandardIRI, a, pmdco.Metadata))
  g.add((extensometerStandardIRI, pmdco.value, Literal(extensometerStandard, datatype=XSD.string)))
  g.add((extensometerIRI, pmdco.characteristic, extensometerStandardIRI))

  micrometerGaugeIRI = URIRef(prefix + "_micrometer_gauge")
  g.add((micrometerGaugeIRI, a, pmdco.MicrometerGauge))
  g.add((micrometerGaugeIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.characteristic, micrometerGaugeIRI))

  micrometerGaugeManufacturerIRI = URIRef(prefix + "_micrometerGaugeManufacturer_" + micrometerGaugeManufacturer)
  g.add((micrometerGaugeManufacturerIRI, a, pmdco.Manufacturer))
  g.add((micrometerGaugeManufacturerIRI, a, pmdco.Metadata))
  g.add((micrometerGaugeManufacturerIRI, pmdco.value, Literal(micrometerGaugeManufacturer, datatype=XSD.string)))
  g.add((micrometerGaugeIRI, pmdco.characteristic, micrometerGaugeManufacturerIRI))

  caliperIRI = URIRef(prefix + "_caliper")
  g.add((caliperIRI, a, pmdco.Caliper))
  g.add((caliperIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.characteristic, caliperIRI))

  caliperManufacturerIRI = URIRef(prefix + "_caliperManufacturer_" + caliperManufacturer)
  g.add((caliperManufacturerIRI, a, pmdco.Manufacturer))
  g.add((caliperManufacturerIRI, a, pmdco.Metadata))
  g.add((caliperManufacturerIRI, pmdco.value, Literal(caliperManufacturer, datatype=XSD.string)))
  g.add((caliperIRI, pmdco.characteristic, caliperManufacturerIRI))

  caliperSerialNumberIRI = URIRef(prefix + "_caliperSerialNumber_" + caliperSerialNumber)
  g.add((caliperSerialNumberIRI, a, pmdco.NodeSerialNumber))
  g.add((caliperSerialNumberIRI, a, pmdco.Metadata))
  g.add((caliperSerialNumberIRI, pmdco.value, Literal(caliperSerialNumber, datatype=XSD.string)))
  g.add((caliperIRI, pmdco.characteristic, caliperSerialNumberIRI))

  loadCellIRI = URIRef(prefix + "_load_cell")
  g.add((loadCellIRI, a, pmdco.LoadCell))
  g.add((loadCellIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.executedBy, loadCellIRI))

  loadCellMaxForceIRI = URIRef(prefix + "_loadCellMaxForce_" + f"{loadCellMaxForce}")
  g.add((loadCellMaxForceIRI, a, pmdco.LoadCellCapacity))
  g.add((loadCellMaxForceIRI, a, pmdco.Metadata))
  g.add((loadCellMaxForceIRI, pmdco.value, Literal(loadCellMaxForce, datatype=XSD.float)))
  g.add((loadCellMaxForceIRI, pmdco.unit, unit.PERCENT))
  g.add((loadCellIRI, pmdco.characteristic, loadCellMaxForceIRI))

  softwareAgentIRI = URIRef(prefix + "_software_agent")
  g.add((softwareAgentIRI, a, prov.SoftwareAgent))
  g.add((softwareAgentIRI, a, pmdco.Metadata))
  g.add((softwareAgentIRI, pmdco.value, Literal("BAM Tensile Test Analysis App", datatype=XSD.string)))
  g.add((processIRI, prov.wasAssociatedWith, softwareAgentIRI))

  # The following section contains some 'bibliographic' information (metadata) using DataCite
  # There is a projectIRI defined already (see above) which we can use here, now.
  BMBF_IRI = URIRef(prefix + "_BMBF")
  g.add((BMBF_IRI, a, datacite.OrganizationIdentifier))
  g.add((BMBF_IRI, datacite.usesIdentifierScheme, datacite.Wikidata))
  g.add((datacite.Wikidata, a, datacite.IdentifierScheme))
  g.add((BMBF_IRI, pmdco.value, Literal("Q492234", datatype=XSD.string)))
  g.add((BMBF_IRI, SKOS.prefLabel, Literal("Bundesministerium für Bildung und Forschung (BMBF)@de", datatype=XSD.string)))
  g.add((BMBF_IRI, SKOS.altLabel, Literal("Federal Ministry of Education and Research@en", datatype=XSD.string)))
  g.add((projectIRI, datacite.hasIdentifier, BMBF_IRI))

  LocalFunderIdentifierSchemeIRI = URIRef(datacite + "_local-funder-identifier-scheme")
  FunderRefIRI = URIRef(prefix + "Funder_Identifier")
  g.add((FunderRefIRI, a, datacite.FunderIdentifier))
  g.add((FunderRefIRI, datacite.usesIdentifierScheme, LocalFunderIdentifierSchemeIRI))
  g.add((LocalFunderIdentifierSchemeIRI, a, datacite.FunderIdentifierScheme))
  g.add((datacite.FunderIdentifierScheme, RDFS.subClassOf, datacite.IdentifierScheme))
  g.add((FunderRefIRI, pmdco.value, Literal("13XP5094", datatype=XSD.string)))
  g.add((projectIRI, datacite.hasIdentifier, FunderRefIRI))


  # break


# Graph Serialization

In [ ]:
# Graph g is serialized in two different data formats / notations, Turtle (ttl) and RDF
g.serialize("RDF-TTO-data_S355.ttl", format="ttl")
g.serialize("RDF-TTO-data_S355.rdf", format='application/rdf+xml', encoding="utf-8-sig")

# Instead of serializing the data to be stored in the working directory, 
# the files could alternatively be stored in another repository or directly
# uploaded to a triple store.

<Graph identifier=Nbc5b63b2514d48c0bc12e1da742bf1f7 (<class 'rdflib.graph.Graph'>)>

# Information concerning Namespaces and Concepts

As some ontologies concepts of which are used within this script do not use IRIs that can directly be interpreted by humans (humans will have to read the labels provided), the corresponding labels are given, here, for the most important concepts frequently used in this script. 

### Information Artifact Ontology 
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:IAO_0020000](http://purl.obolibrary.org/obo/IAO_0020000)       | identifier         | Class |
| [obo:IAO_0000032](http://purl.obolibrary.org/obo/IAO_0000032)       | scalar measurement datum         | Class |
| [obo:IAO_0000003](http://purl.obolibrary.org/obo/IAO_0000003)       | measurement unit label         | Class |
| [obo:IAO_0000027](http://purl.obolibrary.org/obo/IAO_0000027)       | data item         | Class |
| [obo:IAO_0000039](http://purl.obolibrary.org/obo/IAO_0000039)       | has measurement unit label         | Object Property |
| [obo:IAO_0000417](http://purl.obolibrary.org/obo/IAO_0000417)       | is quality measured as         | Object Property |
| [obo:IAO_0000136](http://purl.obolibrary.org/obo/IAO_0000136)       | is about         | Object Property |
| [obo:IAO_0000219](http://purl.obolibrary.org/obo/IAO_0000219)       | denotes         | Object Property |


### Ontology for Biomedical Investigations 
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:OBI_0000011](http://purl.obolibrary.org/obo/OBI_0000011)       | planned process         | Class |
| [obo:OBI_0000070](http://purl.obolibrary.org/obo/OBI_0000070)       | assay         | Class |
| [obo:OBI_0001931](http://purl.obolibrary.org/obo/OBI_0001931)       | scalar value specification         | Class |
| [obo:OBI_0000260](http://purl.obolibrary.org/obo/OBI_0000260)       | plan         | Class |
| [obo:OBI_0000104](http://purl.obolibrary.org/obo/IAO_0000104)       | plan specification        | Class |
| [obo:OBI_0001937](http://purl.obolibrary.org/obo/OBI_0001937)       | has specified numeric value         | Object Property |
| [obo:OBI_0000293](http://purl.obolibrary.org/obo/OBI_0000293)       | has specified input         | Object Property |
| [obo:OBI_0000299](http://purl.obolibrary.org/obo/OBI_0000299)       | has specified output         | Object Property |
| [obo:OBI_0001927](http://purl.obolibrary.org/obo/OBI_0001927)       | specifies value of         | Object Property |
| [obo:OBI_0001938](http://purl.obolibrary.org/obo/OBI_0001938)       | has value specification         | Object Property |



 ### Relation Ontology 
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:RO_0000086](http://purl.obolibrary.org/obo/RO_0000086)       | has quality         | Object Property |
| [obo:RO_0002090](http://purl.obolibrary.org/obo/RO_0002090)       | immediately precedes         | Object Property |
| [obo:RO_0002012](http://purl.obolibrary.org/obo/RO_0002012)       | occurrent part of         | Object Property |
| [obo:RO_0002224](http://purl.obolibrary.org/obo/RO_0002224)       | starts with         | Object Property |
| [obo:RO_0001000](http://purl.obolibrary.org/obo/RO_0001000)       | derives from        | Object Property |
| [obo:RO_0000059](http://purl.obolibrary.org/obo/RO_0000059)       | concretizes        | Object Property |
| [obo:RO_0002180](http://purl.obolibrary.org/obo/RO_0002180)       | has component        | Object Property |
| [obo:RO_0000085](http://purl.obolibrary.org/obo/RO_0000085)       | has function        | Object Property |
| [obo:RO_0002215](http://purl.obolibrary.org/obo/RO_0002215)       | capable of        | Object Property |
| [obo:RO_0002608](http://purl.obolibrary.org/obo/RO_0002608)       | process has causal agent       | Object Property |


### Basic Formal Ontology
*(Namespace: [obo:](http://purl.obolibrary.org/obo/)):*

| IRI  | Label | Concept |
|:-------------|:--------------|:--------------|
| [obo:BFO_0000063](http://purl.obolibrary.org/obo/BFO_0000063)       | precedes         | Object Property |
| [obo:BFO_0000055](http://purl.obolibrary.org/obo/BFO_0000055)       | realizes         | Object Property |

